In [1]:
from openai import AzureOpenAI
from colorama import Fore, Back, Style
from dotenv import load_dotenv
import os
import time
import json
from tools.search_client import AzureSearch

load_dotenv()

openai_client = AzureOpenAI(
                    api_key=os.environ.get("AZURE_OPENAI_API_KEY"), 
                    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
                    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT")
                    )

search_client: AzureSearch = AzureSearch()  # get instance of search to query corpus

init in search client with embedding model: text-embedding-ada-002


In [2]:
# Function to print messages from a thread
def get_messages_from_thread(thread_id):
        messages = openai_client.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            if msg.role == "assistant":
                #print(f"{msg.thread_id}:  {msg.role}: {msg.content[0].text.value}")
                return msg.content[0].text.value # return just the text response to the query

In [3]:
# Function to wait for a run to complete
def wait_for_run_completion(thread_id, run_id):
        while True:
            run = openai_client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            print(f"Current run status: {run.status}")
            time.sleep(2)
            if run.status in ['completed', 'failed', 'requires_action']:
                return run

In [4]:
# Function to perform a Shadow Search
def azure_search(query):
    search_result = search_client.search_hybrid(query)
    #print(search_result)
    return search_result

In [10]:
# Function to handle tool output submission
def submit_tool_outputs(thread_id, run_id, tools_to_call):
        tool_output_array = []
        for tool in tools_to_call:
            output = None
            tool_call_id = tool.id
            function_name = tool.function.name
            function_args = tool.function.arguments

            if function_name == "azure_search":
                print(Fore.GREEN + f"make call to azure_search {json.loads(function_args)['query']}")
                print(Style.RESET_ALL)
                output = azure_search(query=json.loads(function_args)["query"])

            #elif function_name == "profile_prospect":
                #print(Fore.GREEN + f"make call to profile_prospect {json.loads(function_args)['prospect']} - {json.loads(function_args)['context']} ")
                #print(Style.RESET_ALL)
                #output = profile_prospect(prospect=json.loads(function_args)["prospect"], context=json.loads(function_args)["context"])

            #elif function_name == "process_search_results":
            #    print(f"make call to process_search_results {json.loads(function_args)['search_results']}")
            #    output = self.process_search_results(search_results=json.loads(function_args)["search_results"])
            
            if output:
                print(Fore.CYAN + f"[function result] Appending tool output array... \n\n{output}")
                print(Style.RESET_ALL)
                tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

        return openai_client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread_id,
            run_id=run_id,
            tool_outputs=tool_output_array
        )

In [6]:
def chatbot(query: str, thread_id: str):

        # Retrieve an existing assistant which is Shadow Assistant
        assistant = openai_client.beta.assistants.retrieve(
                        assistant_id="asst_YifvR0wR6H41c3cNdSJeO3b3",
                        )  
        
        openai_client.beta.threads.messages.create(  # create a message on the thread that is a user message
                    thread_id=thread_id, 
                    role="user",
                    content=query
                    )
        while True:
            try:
                run = openai_client.beta.threads.runs.create(  # create a run of the thread
                        #additional_instructions="The USER works for the company Frontline",
                        thread_id=thread_id,
                        assistant_id=assistant.id,
                        )
                run = wait_for_run_completion(thread_id, run.id)  # wait for the completion of the run which should return the run
        
                if run.status == 'failed':
                    print(f"Error in run:  {run.failed_at}")
                    continue
                elif run.status == 'requires_action':
                    run = submit_tool_outputs(thread_id, run.id, run.required_action.submit_tool_outputs.tool_calls)
                    run = wait_for_run_completion(thread_id, run.id)

                # Print messages from the thread
                response = get_messages_from_thread(thread_id)
                #print(f"Response:  {response}")

                return response
            except Exception as yikes:
                print(f'\n\nError communicating with AzureOpenAI: "{yikes}"')

In [13]:
thread = openai_client.beta.threads.create()   # create a thread
print(f"Creating new thread: {thread.id}")
query = "Describe the Challenger sales methodology"
r = chatbot(query, thread.id)

print(Fore.YELLOW + f"\n{r}")
print(Style.RESET_ALL)

Creating new thread: thread_MmM7OeYAfc2twkenRLgIxHcA
Current run status: queued
Current run status: requires_action
make call to azure_search Challenger sales methodology

[function result] Appending tool output array... 

[CATEGORY:  Challenger Narration] [SOURCEFILE:  The Challenger exporting the model to the core.txt]Chapter 3. The Challenger exporting the model to the core. Challenger is defined by the ability to do three things. Teach Taylor and take control. And to do all of this through the use of constructive tension. These are the pillars of what we call the Challenger selling model. An approach to sales that is based on what challengers do. It's a methodology that we've worked on with companies across a wide range of industries. Companies as diverse as Talecris, Biotherapeutics, PMI, Brinks in the solutions business of Thomson Reuters. To implement within their own sales organizations. It's previous on the notion that with the right training, coaching and sales tools, most re